<a href="https://colab.research.google.com/github/MLvtt/Dog-Years-Age-Classification/blob/main/Copy_of_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/data/Expert_Train.zip
!unzip /content/drive/MyDrive/data/PetFinder_All.zip

In [ ]:
!pip install split_folders
import splitfolders
splitfolders.ratio('/content/Expert_TrainEval', output='/content/Expert_Split', seed=1337, ratio=(.8, 0.15,0.05))
splitfolders.ratio('/content/PetFinder_All', output='/content/PetFinder_Split', seed=1337, ratio=(.8, 0.15,0.05))
splitfolders.ratio('/content/Expert_TrainEval', output='/content/Combo_Dataset', seed=1337, ratio=(.8, 0.15,0.05))
splitfolders.ratio('/content/PetFinder_All', output='/content/Combo_Dataset', seed=1337, ratio=(.8, 0.15,0.05))

Copying files: 1088 files [00:00, 1501.23 files/s]
Copying files: 26190 files [00:09, 2832.14 files/s]
Copying files: 1088 files [00:03, 355.97 files/s]
Copying files: 26190 files [00:16, 1629.81 files/s]


In [ ]:
img_width = 224
img_height = 224
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling (we do not want to modify the testing data)
val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    '/content/Expert_Split/train/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    '/content/Expert_Split/val/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 870 images belonging to 3 classes.
Found 162 images belonging to 3 classes.


In [ ]:
data_gen = keras.preprocessing.image.ImageDataGenerator( rescale=1. / 255)
data_generator = train_datagen.flow_from_directory('/content/Expert_TrainEval',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1088 images belonging to 3 classes.


In [ ]:
from custom_keras_metrics import *

In [ ]:
IMG_LEN = 224
IMG_SHAPE = (IMG_LEN,IMG_LEN,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
 
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(3, activation='softmax')
])
 
model.compile(optimizer=tf.keras.optimizers.Adamax(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_m, precision_m, recall_m])

In [ ]:
m = model.fit(train_generator, validation_data=validation_generator, verbose=True, epochs=30)

Epoch 1/30
28/28 [==============================] - 41s 1s/step - loss: 1.5009 - accuracy: 0.3782 - f1_m: 0.3617 - precision_m: 0.4004 - recall_m: 0.3337 - val_loss: 1.1569 - val_accuracy: 0.4383 - val_f1_m: 0.4768 - val_precision_m: 0.5401 - val_recall_m: 0.4323
Epoch 2/30
28/28 [==============================] - 36s 1s/step - loss: 0.9537 - accuracy: 0.5437 - f1_m: 0.4888 - precision_m: 0.5909 - recall_m: 0.4211 - val_loss: 1.1567 - val_accuracy: 0.4383 - val_f1_m: 0.3490 - val_precision_m: 0.3967 - val_recall_m: 0.3125
Epoch 3/30
28/28 [==============================] - 36s 1s/step - loss: 0.8599 - accuracy: 0.6080 - f1_m: 0.5429 - precision_m: 0.6666 - recall_m: 0.4628 - val_loss: 1.1646 - val_accuracy: 0.4198 - val_f1_m: 0.2740 - val_precision_m: 0.3686 - val_recall_m: 0.2188
Epoch 4/30
28/28 [==============================] - 36s 1s/step - loss: 0.8653 - accuracy: 0.6103 - f1_m: 0.5838 - precision_m: 0.6744 - recall_m: 0.5182 - val_loss: 1.1677 - val_accuracy: 0.4444 - val_f1_m: 

In [ ]:
acc = m.history['accuracy']
val_acc = m.history['val_accuracy']
f1 = m.history['f1_m']
val_f1 = m.history['val_f1_m']
prc = m.history['precision_m']
val_prc = m.history['val_precision_m']
rcl = m.history['recall_m']
val_rcl = m.history['val_recall_m']
loss = m.history['loss']
val_loss = m.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(4, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(4, 1, 1)
plt.plot(f1, label='Training F1')
plt.plot(val_f1, label='Validation F1')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation F1')
plt.xlabel('epoch')
plt.show()

plt.subplot(4, 2, 2)
plt.plot(, label='Training F1')
plt.plot(val_f1, label='Validation F1')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation F1')
plt.xlabel('epoch')
plt.show()

plt.subplot(4, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

SyntaxError: ignored